#### Importing

In [188]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import re
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_percentage_error

from scipy.stats import randint, uniform

import lightgbm as lgb

In [189]:
train = pd.read_csv("./data/train.csv")
train_y = train[["SALE PRICE"]]
low = train_y['SALE PRICE'].quantile(0.01)
up = train_y['SALE PRICE'].quantile(0.99)
train = train[(train_y['SALE PRICE'] > low) & (train_y['SALE PRICE'] < up)]
train_X = train
train_y = train[["SALE PRICE"]]
del train_X["SALE PRICE"]

train_y_log = np.log1p(train_y)

test_X = pd.read_csv("./data/test.csv")
test_y = pd.read_csv("./data/test_groundtruth.csv")

print("train_X:",train_X.shape)
print("train_y:",train_y.shape)
print("test_X:",test_X.shape)
print("test_y:",test_y.shape)

train_X: (42175, 19)
train_y: (42175, 1)
test_X: (10767, 19)
test_y: (10767, 1)


#### Original Dataset

In [190]:
num_train_samples = len(train_X)

data_X = pd.concat([train_X, test_X])
data_X.head(4)

,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,BUILDING CLASS AT PRESENT,ADDRESS,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE DATE
0,1,UPPER EAST SIDE (79-96),13 CONDOS - ELEVATOR APARTMENTS,2,1569,1027,R4,402 EAST 90TH STREET,5A,10128,1,0,1,-,-,1910,2,R4,2016-09-30 00:00:00
2,3,DOWNTOWN-FULTON MALL,13 CONDOS - ELEVATOR APARTMENTS,2,170,1042,R4,265 STATE STREET,910,11201,1,0,1,0,0,2014,2,R4,2017-01-25 00:00:00
3,4,FLUSHING-NORTH,13 CONDOS - ELEVATOR APARTMENTS,2,4410,1023,R4,137-11 32 AVENUE,4W,11354,1,0,1,-,-,0,2,R4,2017-03-17 00:00:00
4,3,PARK SLOPE,09 COOPS - WALKUP APARTMENTS,2C,1067,29,C6,"862 PRESIDENT STREET, 1",,11215,0,0,0,0,0,1920,2,C6,2016-09-09 00:00:00


### Encoding and Data Pre Processing

In [ ]:
#del data_X['ADDRESS']
#del data_X['APARTMENT NUMBER']
#del data_X['BUILDING CLASS AT PRESENT']
#del data_X['BUILDING CLASS AT TIME OF SALE']
#del data_X['NEIGHBORHOOD']
del data_X['SALE DATE']
#del data_X['LAND SQUARE FEET']
#del data_X['GROSS SQUARE FEET']

In [192]:
need_encoding = data_X.columns
need_1hot = list(i for i in need_encoding if i in [
    'BOROUGH',
    'NEIGHBORHOOD',
    'BUILDING CLASS CATEGORY',
    'BUILDING CLASS AT PRESENT',
    'TAX CLASS AT PRESENT',
    'BUILDING CLASS AT TIME OF SALE',
])
need_freq = list(i for i in need_encoding if i in ['BLOCK', 'LOT', 'APARTMENT NUMBER'])
need_fill = list(i for i in need_encoding if i in ['LAND SQUARE FEET', 'GROSS SQUARE FEET', 'YEAR BUILT'])
need_log = list(i for i in need_encoding if i in ['LAND SQUARE FEET', 'GROSS SQUARE FEET'])

    address

In [193]:
if 'ADDRESS' in need_encoding:
    street_suffix_map = {
        'STREET': ['ST', 'STREET'],
        'AVENUE': ['AVE', 'AVENUE', 'AV', 'AVN'],
        'ROAD': ['RD', 'ROAD'],
        'DRIVE': ['DR', 'DRIVE', 'DRV'],
        'LANE': ['LN', 'LANE'],
        'BOULEVARD': ['BLVD', 'BOUL', 'BOULV'],
        'COURT': ['CT', 'COURT'],
        'PLACE': ['PL', 'PLACE'],
        'SQUARE': ['SQ', 'SQUARE'],
        'TERRACE': ['TER', 'TERRACE'],
        'WAY': ['WAY'],
        'CIRCLE': ['CIR', 'CIRCLE', 'CRCL'],
        'HIGHWAY': ['HWY', 'HIGHWAY'],
        'PARKWAY': ['PKWY', 'PARKWAY', 'PKY'],
        'ALLEY': ['ALY', 'ALLEY'],
        'TRAIL': ['TRL', 'TRAIL', 'TR']
    }

    
    all_suffixes_flat = []
    for standard_form, variants in street_suffix_map.items():
        for variant in variants:
            all_suffixes_flat.append(variant)
    all_suffixes_flat.sort(key=len, reverse=True)

    regex_pattern = r"(?i)\b(" + "|".join(re.escape(s) for s in all_suffixes_flat) + r")\b"

    temp_address_series = data_X['ADDRESS'].astype(str)
    extracted_suffixes = temp_address_series.str.extract(regex_pattern, expand=False)
    data_X['EXTRACTED_SUFFIX_RAW'] = extracted_suffixes.str.upper()

    variant_to_standard_map = {}
    for standard, variants in street_suffix_map.items():
        for variant in variants:
            variant_to_standard_map[variant.upper()] = standard

    data_X['STREET_SUFFIX_STANDARD'] = data_X['EXTRACTED_SUFFIX_RAW'].map(variant_to_standard_map)
    data_X['STREET_SUFFIX_STANDARD'] = data_X['STREET_SUFFIX_STANDARD'].fillna('OTHER_SUFFIX')

    data_X['STREET_SUFFIX_STANDARD'] = data_X['STREET_SUFFIX_STANDARD'].astype('category')
    one_hot_suffixes = pd.get_dummies(data_X['STREET_SUFFIX_STANDARD'], prefix='Suffix', dtype=int)
    data_X = pd.concat([data_X, one_hot_suffixes], axis=1)

    if 'EXTRACTED_SUFFIX_RAW' in data_X.columns:
        del data_X['EXTRACTED_SUFFIX_RAW']
    
    del data_X['STREET_SUFFIX_STANDARD']
    del data_X['ADDRESS']

    

    timing

In [194]:
if 'SALE DATE' in need_encoding:
    data_X['SALE YEAR'] = pd.to_datetime(data_X['SALE DATE']).dt.year
    data_X['SALE MONTH'] = pd.to_datetime(data_X['SALE DATE']).dt.month
    data_X['SALE DATE'] = pd.to_datetime(data_X['SALE DATE']).dt.day

    1-hot

In [ ]:

data_X[need_1hot] = data_X[need_1hot].astype('category')

one_hot_encoded = pd.get_dummies(data_X[need_1hot], dtype=int)

data_X = data_X.drop(need_1hot,axis=1)
data_X = pd.concat([data_X, one_hot_encoded] ,axis=1)

    filling

In [196]:
### 注意!编码只能用训练数据算mean!!!!!

data_X[need_fill] = data_X[need_fill].apply(lambda x: pd.to_numeric(x, errors='coerce'))
train[need_fill] = train[need_fill].apply(lambda x: pd.to_numeric(x, errors='coerce'))
data_X[need_fill] = data_X[need_fill].replace([0], np.nan)
mean = train[need_fill].mean(skipna=True)
data_X[need_fill] = data_X[need_fill].fillna(mean)

    freq encoding

In [197]:
### 注意!编码只能用训练数据算mean!!!!!

for i in need_freq:
    freq_map = train[i].value_counts(normalize=True)
    data_X[i] = data_X[i].map(freq_map)
    data_X[i] = data_X[i].fillna(0) 

    logging

In [ ]:
#for i in need_log:
#    data_X[i] = np.log(data_X[i])

#### Last Checking and Scaling

In [199]:
data_X.head(10)

,BLOCK,LOT,APARTMENT NUMBER,ZIP CODE,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,...,BUILDING CLASS AT TIME OF SALE_V1,BUILDING CLASS AT TIME OF SALE_V2,BUILDING CLASS AT TIME OF SALE_V9,BUILDING CLASS AT TIME OF SALE_W2,BUILDING CLASS AT TIME OF SALE_W3,BUILDING CLASS AT TIME OF SALE_W4,BUILDING CLASS AT TIME OF SALE_W8,BUILDING CLASS AT TIME OF SALE_W9,BUILDING CLASS AT TIME OF SALE_Z0,BUILDING CLASS AT TIME OF SALE_Z9
0,0.000403,0.001043,0.002371,10128,1,0,1,7.733249,7.375283,1910.000000,...,0,0,0,0,0,0,0,0,0,0
2,0.002229,0.000877,0.000047,11201,1,0,1,7.733249,7.375283,2014.000000,...,0,0,0,0,0,0,0,0,0,0
3,0.000166,0.001304,0.000142,11354,1,0,1,7.733249,7.375283,1825.068382,...,0,0,0,0,0,0,0,0,0,0
4,0.000522,0.009840,0.782573,11215,0,0,0,7.733249,7.375283,1920.000000,...,0,0,0,0,0,0,0,0,0,0
5,0.000142,0.005714,0.003770,11238,1,0,1,7.733249,7.375283,1825.068382,...,0,0,0,0,0,0,0,0,0,0
6,0.000071,0.009721,0.782573,11435,0,0,0,7.733249,7.375283,1954.000000,...,0,0,0,0,0,0,0,0,0,0
7,0.000071,0.055459,0.782573,10306,0,0,0,8.699515,7.375283,1825.068382,...,0,0,0,0,0,0,0,0,0,0
8,0.000047,0.007374,0.782573,11417,1,0,1,7.600902,7.445418,1920.000000,...,0,0,0,0,0,0,0,0,0,0
9,0.000213,0.000640,0.000190,10038,1,0,1,7.733249,7.375283,1919.000000,...,0,0,0,0,0,0,0,0,0,0
10,0.000095,0.004102,0.001446,11218,3,0,3,7.602401,8.342840,1931.000000,...,0,0,0,0,0,0,0,0,0,0


In [200]:
data_X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52942 entries, 0 to 10766
Columns: 563 entries, BLOCK to BUILDING CLASS AT TIME OF SALE_Z9
dtypes: float64(6), int32(3), int64(554)
memory usage: 227.2 MB


In [201]:
data_X.columns

Index(['BLOCK', 'LOT', 'APARTMENT NUMBER', 'ZIP CODE', 'RESIDENTIAL UNITS',
       'COMMERCIAL UNITS', 'TOTAL UNITS', 'LAND SQUARE FEET',
       'GROSS SQUARE FEET', 'YEAR BUILT',
       ...
       'BUILDING CLASS AT TIME OF SALE_V1',
       'BUILDING CLASS AT TIME OF SALE_V2',
       'BUILDING CLASS AT TIME OF SALE_V9',
       'BUILDING CLASS AT TIME OF SALE_W2',
       'BUILDING CLASS AT TIME OF SALE_W3',
       'BUILDING CLASS AT TIME OF SALE_W4',
       'BUILDING CLASS AT TIME OF SALE_W8',
       'BUILDING CLASS AT TIME OF SALE_W9',
       'BUILDING CLASS AT TIME OF SALE_Z0',
       'BUILDING CLASS AT TIME OF SALE_Z9'],
      dtype='object', length=563)

In [202]:
train_X = data_X[:num_train_samples].to_numpy()
test_X = data_X[num_train_samples:].to_numpy()
train_y_log = train_y_log.to_numpy().ravel()

In [203]:
scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_X) 
test_X_scaled = scaler.transform(test_X) 

# Regression

In [204]:
M = RandomForestRegressor(
    n_estimators=200,
    criterion='squared_error',
    max_depth=25,
    min_samples_split=2,
    min_samples_leaf=3,
    min_weight_fraction_leaf=0.0,
    max_features=0.3,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=True,
    n_jobs=-1,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
    monotonic_cst=None
)

'''M = lgb.LGBMRegressor(
    boosting_type='rf',
    bagging_freq=1,
    bagging_fraction=0.9,
    num_leaves=1000,
    max_depth=-1,
    learning_rate=0.1,
    n_estimators=2000,
    subsample_for_bin=200000,
    objective='mape',
    class_weight=None,
    min_split_gain=0.0,
    min_child_weight=0.001,
    min_child_samples=20,
    subsample=1,
    subsample_freq=1,
    colsample_bytree=1.0,
    reg_alpha=0.0,
    reg_lambda=0.1,
    random_state=None,
    n_jobs=8,
    importance_type='split'
)'''

param = {
    'n_estimators': [100, 200],        
    'max_depth': [15, 25],            
    'min_samples_leaf': [3, 4, 5],         
    'max_features': ['sqrt', 0.3]      
}

grid_search = GridSearchCV(
    estimator=M,           
    param_grid=param,             
    cv=3,                              
    scoring='neg_mean_squared_error',  
    n_jobs=3,                         
    verbose=2                     
)

#print('GridSearchCV working on RF...')
#grid_search.fit(train_X_scaled, train_y_log)
#print('completed')
#print(f"best param: {grid_search.best_params_}")
#M = grid_search.best_estimator_

M.fit(train_X_scaled, train_y_log)

RandomForestRegressor(max_depth=25, max_features=0.3, min_samples_leaf=3,
                      n_estimators=200, n_jobs=-1, oob_score=True)

In [205]:
import gc

del grid_search

gc.collect()

123

In [206]:
N = MLPRegressor(
    hidden_layer_sizes=(150, 100, 50),
    activation='relu',               
    solver='adam',                   
    alpha=0.0082, 
    batch_size=64,
    learning_rate='adaptive',     
    learning_rate_init=0.001, 
    max_iter=1000,
    shuffle=True,   
    random_state=42,  
    early_stopping=True,
    validation_fraction=0.1, 
    n_iter_no_change=15,
    verbose=False
)

param = {
    'hidden_layer_sizes': [
        (64, 32), 
        (100, 50, 25), 
        (128, 64), 
        (150, 100, 50),
        (200,)
    ],
    'alpha': uniform(0.0001, 0.01),
    'learning_rate_init': [0.001, 0.0005, 0.0001],
    'batch_size': [32, 64, 128, 256]
}

random_search = RandomizedSearchCV(
    estimator=N,                   
    param_distributions=param,    
    n_iter=20,                                  
    cv=3,                                       
    scoring='neg_mean_squared_error',           
    n_jobs=3,                                  
    verbose=2,                                  
    random_state=74                            
)

#print('RandomizedSearchCV working on MLP...')
#random_search.fit(train_X_scaled, train_y_log)
#print('completed')
#print(f"best param: {random_search.best_params_}")
#N = random_search.best_estimator_
N.fit(train_X_scaled, train_y_log)

MLPRegressor(alpha=0.0082, batch_size=64, early_stopping=True,
             hidden_layer_sizes=(150, 100, 50), learning_rate='adaptive',
             max_iter=1000, n_iter_no_change=15, random_state=42)

In [207]:
import gc

del random_search

gc.collect()

0

In [219]:
u = 0.96
train_Y_pre = u * np.expm1(M.predict(train_X_scaled)) - (1 - u) * np.expm1(N.predict(train_X_scaled)) - v
mean_absolute_percentage_error(train_y,train_Y_pre)

0.2227970092004197

In [220]:
Y_pred = u * np.expm1(M.predict(test_X_scaled)) - (1 - u) * np.expm1(N.predict(test_X_scaled)) - v
mean_absolute_percentage_error(test_y,Y_pred)

0.32287382779051454

In [210]:
pd.DataFrame({"pred":Y_pred}).to_csv("1234_Zhuiy.csv")